In [1]:
!pip install PyMuPDF openai
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [7]:
import fitz  # PyMuPDF
import openai

def extract_text_from_pdf(pdf_path):
    # Open the provided PDF file
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text



In [8]:
# Path to your PDF file
pdf_path = "/content/OREA-Form-100 (2)Agreement of Purchase and Sale.pdf"
# here to give path of contract--- example

# Extract text from the PDF
contract_text = extract_text_from_pdf(pdf_path)

In [9]:
contract_text

'Form 100    Revised 2020     Page 1 of 6\nThe trademarks REALTOR®, REALTORS®, MLS®, Multiple Listing Services® and associated logos are owned or controlled by\nThe Canadian Real Estate Association (CREA) and identify the real estate professionals who are members of CREA and the\nquality of services they provide. Used under license.\n© 2020, Ontario Real Estate Association (“OREA”). All rights reserved. This form was developed by OREA for the use and reproduction  \nby its members and licensees only. Any other use or reproduction is prohibited except with prior written consent of OREA. Do not alter  \nwhen printing or reproducing the standard pre-set portion. OREA bears no liability for your use of this form.\nAgreement of Purchase and Sale\nForm 100\nfor use in the Province of Ontario\nThis Agreement of Purchase and Sale dated this ....................... day of .................................................................................................... 20............\nBUYER: 

In [10]:
from openai import OpenAI
API_KEY = ''
### Here to give API
client = OpenAI(
  api_key=API_KEY
 )

In [17]:
def get_template(content,questions,answers=''):
  template = """
  ### Contract Content:
  {}

  ### Questions:
  {}
  ### Answer to each question:
  {}

  """.format(content,'\n'.join(questions),'\n'.join(answers) if answers else '')
  return template

In [39]:
questions = [
    "What is the address of the property?",
    "What is the deadline for the closing?",
    "Is it possible for the buyer or seller to delay the closing by a few days?",
    "Can intrinsic evidence be considered admissible in relation to the contract?",
    "Is the buyer or the seller responsible for fees associated with title transfer?"
]


In [40]:
answers=['Property Address: 1 Main St, New York, New York','Closing Date: January 1, 2022','No. Because time is of the essence.', 'No.','seller OREA section 16']
Example= get_template(contract_text,questions,answers)
Example

'\n  ### Contract Content:\n  Form 100    Revised 2020     Page 1 of 6\nThe trademarks REALTOR®, REALTORS®, MLS®, Multiple Listing Services® and associated logos are owned or controlled by\nThe Canadian Real Estate Association (CREA) and identify the real estate professionals who are members of CREA and the\nquality of services they provide. Used under license.\n© 2020, Ontario Real Estate Association (“OREA”). All rights reserved. This form was developed by OREA for the use and reproduction  \nby its members and licensees only. Any other use or reproduction is prohibited except with prior written consent of OREA. Do not alter  \nwhen printing or reproducing the standard pre-set portion. OREA bears no liability for your use of this form.\nAgreement of Purchase and Sale\nForm 100\nfor use in the Province of Ontario\nThis Agreement of Purchase and Sale dated this ....................... day of ...............................................................................................

In [47]:
responses = []

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "We will give you some contracts and Please answer the following inquiries: ”What is the address of the property?”, ”What is the deadline for the closing?”, ”Is it possible for the buyer or seller to delay the closing by a few days?”, etc. If the infomation is not provided, just imply by law. "},
    {"role":"system","content":f"Here is an example: {Example}"},
    {"role": "user", "content": get_template(contract_text,questions)}
  ],
  max_tokens=200
)

responses.append(completion.choices[0].message.content)


In [48]:
print(responses[0])

What is the address of the property?  
Answer: The address of the property is not explicitly mentioned in the contract. Typically, the property address is found in the section titled "REAL PROPERTY" in the agreement.

What is the deadline for the closing?  
Answer: The closing deadline is not explicitly mentioned in the provided contract. However, according to standard real estate practices, the deadline for closing is usually specified in the "COMPLETION DATE" section of the agreement.

Is it possible for the buyer or seller to delay the closing by a few days?  
Answer: In the absence of specific provisions allowing for extension or delay in the contract, time is usually considered of the essence in real estate contracts. This means that closing deadlines are typically expected to be adhered to strictly, and delays by a few days may not be permissible without mutual agreement from both parties.

Can intrinsic evidence be considered admissible in relation to the contract?  
Answer: Int